Conversion between photon cm$^{-2}$ s$^{-1}$ sr$^{-1}$ and erg cm$^{-2}$ s$^{-1}$ arcsec$^{-2}$.

In [2]:
from astropy import constants as const
from astropy import units as u

photon_wavelength = 656.3e-9*u.m

SB_cgs = 10**-22 *u.erg / u.arcsec**2 / u.cm**2 / u.s
SB_ph = SB_cgs * photon_wavelength/(const.h.to('erg s') * const.c.to('m/s')) * (206265.*u.arcsec)**2/u.sr

print('The surface brightness, %s, in cgs units is: %s'%(SB_ph.round(1),SB_cgs))

The surface brightness, 1.4 1 / (cm2 s sr), in cgs units is: 1e-22 erg / (arcsec2 cm2 s)


Conversion between photon/pixel and mag arcsec$^{-2}$ (from Abraham et al. 2014):

>The surface brightness in equation (4), S, can be cast into units of
magnitude per arcsecsecond squared (mag arcsec$^{-2}$) by noting that
$S = F \times 10^{-0.4\mu} $
, where F is the flux zero point for the
filter and $\mu$ is the mean surface brightness in magnitude units in the bandpass.

>In the AB system, the zeroth magnitude calibrator is defined in frequency space as 3631 Jy in all bands, so after conversion back to wavelength space, 
$F = (3631$ Jy/arcsec$^{2}) \times c / \lambda^2$
,where $\lambda$ is the effective wavelength of the filter.

In [7]:
SB_mag = 21.5 #* u.mag / u.arcsec**2
F_zp = 3631. * u.Jy / u.arcsec**2
eff_wavelen = 475. * 10**-9 * u.m # just by eyeballing the G filter transmission spectrum
SB_ph = F_zp * const.c / eff_wavelen**2 * 10**(-0.4*SB_mag)

print('The surface brightness, %s mag/arcsec^2, is: %s'%(SB_mag, SB_ph))

The surface brightness, 21.5 mag/arcsec^2, is: 1.21188031907e+16 Jy / (arcsec2 m s)
